Implemente utilizando Polars los siguientes algoritmos para encontrar reglas de asociación:· 

·        Apriori

·         FP-Growth

Compare ambos algoritmos con el mismo conjunto de datos



## Reglas de asociación

1 {a,b,c}

2 {a,b}

3 {a,c,d}

4 {a}

5 {b,c}

6 {b,c,d}

a continuación el algoritmo para la probabilidad condicional superior al 50% del anterior conjunto de datos


In [ ]:
#pip install polar
import polars as pl

df = pl.DataFrame({
    "a": [1,1,1,1,0,0],
    "b": [1,1,0,0,1,1],
    "c": [1,0,1,0,1,1],
    "d":[0,0,1,0,0,1]
})

def proba(p1,p2):
  pr=df.filter((df[p1]==1) & (df[p2]==1))
  pc=pr.shape[0]/df[p2].sum()
  return "prob de: {"+p1+"} -> {"+p2+"} = "+str(pc)


print(proba("a","b"))
print(proba("a","c"))
print(proba("a","d"))

print(proba("b","a"))
print(proba("b","c"))
print(proba("b","d"))


print(proba("c","a"))
print(proba("c","b"))
print(proba("c","d"))

print(proba("d","a"))
print(proba("d","b"))
print(proba("d","c"))



prob de: {a} -> {b} = 0.5
prob de: {a} -> {c} = 0.5
prob de: {a} -> {d} = 0.5
prob de: {b} -> {a} = 0.5
prob de: {b} -> {c} = 0.75
prob de: {b} -> {d} = 0.5
prob de: {c} -> {a} = 0.5
prob de: {c} -> {b} = 0.75
prob de: {c} -> {d} = 1.0
prob de: {d} -> {a} = 0.25
prob de: {d} -> {b} = 0.25
prob de: {d} -> {c} = 0.5


In [ ]:
#pip install polar
import polars as pl
import time

start_time = time.perf_counter()

df = pl.DataFrame({
    "a": [1,1,0,0,1],
    "b": [0,1,1,1,0],
    "c": [1,1,0,1,1],
    "d": [1,0,0,0,0],
    "e": [0,0,0,0,1],
    "f": [1,1,1,0,1],
    "g": [1,0,0,0,0],
    "h": [0,0,1,0,0],
    "i": [1,0,0,0,0],
    "j": [0,0,1,0,0],
    "k": [0,0,1,0,0],
    "l": [0,1,0,0,1],
    "m": [1,1,0,0,1],
    "o": [0,1,1,0,0],
    "p": [1,0,0,1,1]
})

#función que retorna la cantidad de veces que las letras estan ocurriendo en un mismo vector columna
def proba(letras):
  pr=df.select(letras)
  aa = pr.with_columns((pl.struct(pl.all()).map_elements(lambda row: len(set(row.values())) == 1, return_dtype=pl.Boolean)).alias("igual"))
  bb=aa.filter(aa["igual"])
  return bb.shape[0]


#probabilidad condicionada
numerador=['m']
denominador=['a','c']

print("P("+str(numerador)+'/'+str(denominador)+')='+str(proba(numerador+denominador)/proba(denominador)))

#parte de código que da el tiempo de ejecución
end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.6f} segundos")


P(['m']/['a', 'c'])=1.0
Tiempo de ejecución: 0.005902 segundos


### Tarea Probabilidad condicionada

comparar los dos algoritmos en tiempo y precisión para la probabilidad condicionada, la parte anterior falta por compleemntar la probabilidad condicionada.



```
{f,a,c,d,g,l,m,p}
{a,b,c,f,l,m,o}
{b,f,h,j,o}
{b,c,k,s,p}
{a,f,c,e,l,p,m,n}

```

## Probabilidad Condicionada

La **probabilidad condicionada** se define como la probabilidad de que ocurra un evento **A**, dado que ya ha ocurrido otro evento **B**. Se denota como **P(A | B)** y se calcula con la siguiente fórmula:  

$$ P(A | B) = \frac{P(A \cap B)}{P(B)}$$

### Explicación de la fórmula
- **P(A | B)** → Probabilidad de que ocurra A dado que B ya ocurrió.  
- **P(A ∩ B)** → Probabilidad de que A y B ocurran simultáneamente.  
- **P(B)** → Probabilidad de que ocurra B (siempre que \( P(B) > 0 \)).  

---


In [ ]:
# función para encontrar la probabilidad condicionada

import time
start_time = time.perf_counter()

a=['f','a','c','d','g','i','m','p']
b=['a','b','c','f','l','m','o']
c=['b','f','h','j','o']
d=['b','c','k','s','p']
e=['a','f','c','e','l','p','m','n']

#función que concatena todas las listas definidas para encontrar su frecuencia
def concatenar(vec):
  return [item for sublist in vec for item in sublist]

union=concatenar([a,b,c,d,e])

#se obtiene la frecuencia de aparición de cada letra dentro de los listados definidos
counts={}
for item in union:
    counts[item] = counts.get(item, 0) + 1

print('Frecuencia de letras: '+str(counts))

#función que retorna el nuevo listado de palabras de acuerdo con su frecuencia de aparición
def nueva_lista(lista,delta):
  nueva_lista=[]
  for i in range(len(lista)):
    if counts[lista[i]]>delta:
      nueva_lista.append(lista[i])
  return nueva_lista

a1=nueva_lista(a,2)
b1=nueva_lista(b,2)
c1=nueva_lista(c,2)
d1=nueva_lista(d,2)
e1=nueva_lista(e,2)

listado=[a1,b1,c1,d1,e1]

print("Nuevos arreglos:"+str(listado))

#función que retorna la 'probabilidad' o cuenta de aparición de cada letra o conjunto de letras
def prob(letras,listado):
  cuenta=0
  for i in range(len(listado)):
    res=all(item in listado[i] for item in letras)
    if res:
      cuenta+=1
  return cuenta

#probabilidad condicionada
numerador=['m']
denominador=['a','c']

print("P("+str(numerador)+'/'+str(denominador)+')='+str(prob(numerador+denominador,listado)/prob(denominador,listado)))


end_time = time.perf_counter()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.6f} segundos")

Frecuencia de letras: {'f': 4, 'a': 3, 'c': 4, 'd': 1, 'g': 1, 'i': 1, 'm': 3, 'p': 3, 'b': 3, 'l': 2, 'o': 2, 'h': 1, 'j': 1, 'k': 1, 's': 1, 'e': 1, 'n': 1}
Nuevos arreglos:[['f', 'a', 'c', 'm', 'p'], ['a', 'b', 'c', 'f', 'm'], ['b', 'f'], ['b', 'c', 'p'], ['a', 'f', 'c', 'p', 'm']]
P(['m']/['a', 'c'])=1.0
Tiempo de ejecución: 0.003655 segundos
